# Tutorial 2: Foundation Models Showcase

This tutorial demonstrates how to use NeurOS's foundation models for neural decoding.

**Foundation Models Covered:**
1. POYO+ (Multi-task decoding)
2. NDT3 (Motor decoding)
3. CEBRA (Latent embeddings)
4. Neuroformer (Zero-shot learning)

**Prerequisites:**
- Basic Python knowledge
- Familiarity with NumPy
- Tutorial 1 completed (recommended)

## Setup and Imports

In [ ]:
# Install NeurOS if needed
# !pip install neuros[foundation]

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, r2_score

# NeurOS imports
from neuros.foundation_models import (
    POYOPlusModel,
    NDT3Model,
    CEBRAModel,
    NeuroformerModel
)
from neuros.datasets import load_allen_mock_data

# Set random seed for reproducibility
np.random.seed(42)

print("✅ Imports successful!")

## 1. Load Neural Data

We'll use mock Allen Institute-style data for this tutorial.
In practice, you'd load real data from NWB files or Allen SDK.

In [ ]:
# Load mock neural data
data = load_allen_mock_data(n_neurons=100, duration=120.0)

print(f"Dataset info:")
print(f"  - Neurons: {data['n_neurons']}")
print(f"  - Duration: {data['duration']}s")
print(f"  - Spike trains: {len(data['spike_times'])}")
print(f"  - Spike raster shape: {data['spike_raster'].shape}")

# Extract features for models
X_neural = data['spike_raster']  # (n_trials, n_neurons, n_bins)
y_behavior = np.random.randint(0, 3, size=X_neural.shape[0])  # Mock labels

print(f"\nInput data shape: {X_neural.shape}")
print(f"Labels shape: {y_behavior.shape}")

## 2. POYO+ Model (Multi-Task Decoding)

POYO+ can perform multiple decoding tasks simultaneously from the same neural data.
It's pretrained on large-scale datasets and can be fine-tuned for your specific tasks.

In [ ]:
# Create POYO+ model with multiple tasks
task_configs = [
    {"name": "behavior", "type": "classification", "output_dim": 3},
    {"name": "position", "type": "regression", "output_dim": 2},
]

poyo = POYOPlusModel(task_configs=task_configs)

# Train on your data
print("Training POYO+ model...")
poyo.train(X_neural[:80], y_behavior[:80])

# Make predictions (returns dict with all tasks)
predictions = poyo.predict(X_neural[80:])

print(f"\nPOYO+ Predictions:")
for task_name, preds in predictions.items():
    print(f"  {task_name}: shape {preds.shape}")

# Evaluate
behavior_preds = predictions['behavior']
behavior_labels = np.argmax(behavior_preds, axis=1)
accuracy = accuracy_score(y_behavior[80:], behavior_labels)
print(f"\nBehavior classification accuracy: {accuracy:.2%}")

### Visualize Multi-Task Predictions

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Task 1: Behavior classification
axes[0].bar(range(3), np.bincount(behavior_labels, minlength=3))
axes[0].set_xlabel('Class')
axes[0].set_ylabel('Count')
axes[0].set_title('POYO+ Behavior Classification')
axes[0].set_xticks(range(3))

# Task 2: Position prediction
position_preds = predictions['position']
axes[1].scatter(position_preds[:, 0], position_preds[:, 1], 
                c=behavior_labels, cmap='viridis', alpha=0.6)
axes[1].set_xlabel('Position X')
axes[1].set_ylabel('Position Y')
axes[1].set_title('POYO+ Position Decoding')
axes[1].colorbar(label='Behavior Class')

plt.tight_layout()
plt.show()

## 3. NDT3 Model (Motor Decoding)

NDT3 is optimized for real-time motor decoding from intracortical recordings.
It features cross-subject transfer and rapid fine-tuning.

In [ ]:
# Create NDT3 model for motor decoding
ndt3 = NDT3Model(
    n_neurons=100,
    output_dim=2,  # 2D motor output (e.g., cursor velocity)
    use_subject_embedding=True
)

# Mock motor outputs (velocity in x, y)
motor_outputs = np.random.randn(X_neural.shape[0], 2)

# Train model
print("Training NDT3 model...")
ndt3.train(X_neural[:80], motor_outputs[:80], subject_ids=None)

# Predict motor outputs
motor_preds = ndt3.predict(X_neural[80:])
print(f"\nMotor predictions shape: {motor_preds.shape}")

# Evaluate
r2_x = r2_score(motor_outputs[80:, 0], motor_preds[:, 0])
r2_y = r2_score(motor_outputs[80:, 1], motor_preds[:, 1])
print(f"\nMotor Decoding R² Scores:")
print(f"  X-axis: {r2_x:.3f}")
print(f"  Y-axis: {r2_y:.3f}")

### Fine-Tuning for New Subject

In [ ]:
# Simulate new subject data (only 10 samples)
X_new_subject = X_neural[-10:]
y_new_subject = motor_outputs[-10:]

# Fine-tune on minimal data
print("Fine-tuning NDT3 for new subject...")
ndt3.fine_tune(
    X_new_subject,
    y_new_subject,
    subject_id=99,
    n_epochs=5
)

# Predict with adapted model
adapted_preds = ndt3.predict(X_new_subject, subject_id=99)

# Compare before/after adaptation
r2_adapted = r2_score(y_new_subject[:, 0], adapted_preds[:, 0])
print(f"\nR² after fine-tuning: {r2_adapted:.3f}")
print("✅ NDT3 successfully adapted to new subject!")

## 4. CEBRA Model (Latent Embeddings)

CEBRA learns consistent, low-dimensional representations of neural activity
that capture behavioral correlates.

In [ ]:
# Reshape data for CEBRA (expects 2D: samples x features)
X_2d = X_neural.reshape(X_neural.shape[0], -1)

# Create CEBRA model
cebra = CEBRAModel(
    input_dim=X_2d.shape[1],
    output_dim=3,  # 3D latent space for visualization
    learning_mode='time',  # Can also use 'behavior' or 'hybrid'
    temperature=0.1
)

# Fit and transform (sklearn-style API)
print("Training CEBRA model...")
embeddings = cebra.fit_transform(X_2d)

print(f"\nCEBRA Embeddings shape: {embeddings.shape}")
print(f"Embedding space: {embeddings.shape[1]}D")

### Visualize Latent Space

In [ ]:
fig = plt.figure(figsize=(12, 5))

# 2D projection
ax1 = fig.add_subplot(121)
scatter = ax1.scatter(
    embeddings[:, 0], 
    embeddings[:, 1],
    c=y_behavior,
    cmap='viridis',
    alpha=0.6,
    s=50
)
ax1.set_xlabel('CEBRA Dimension 1')
ax1.set_ylabel('CEBRA Dimension 2')
ax1.set_title('CEBRA Latent Space (2D)')
plt.colorbar(scatter, ax=ax1, label='Behavior')

# 3D projection
ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(
    embeddings[:, 0],
    embeddings[:, 1],
    embeddings[:, 2],
    c=y_behavior,
    cmap='viridis',
    alpha=0.6,
    s=50
)
ax2.set_xlabel('Dim 1')
ax2.set_ylabel('Dim 2')
ax2.set_zlabel('Dim 3')
ax2.set_title('CEBRA Latent Space (3D)')

plt.tight_layout()
plt.show()

### Cross-Session Consistency

In [ ]:
# Simulate second session
data_session2 = load_allen_mock_data(n_neurons=100, duration=120.0)
X_session2 = data_session2['spike_raster'].reshape(data_session2['spike_raster'].shape[0], -1)

# Compute cross-session consistency
consistency = cebra.compute_consistency(
    X_2d[:50], 
    X_session2[:50],
    n_neighbors=5
)

print(f"\nCross-session consistency: {consistency:.3f}")
print("(Higher is better, range 0-1)")

## 5. Neuroformer Model (Zero-Shot & Few-Shot)

Neuroformer is a multimodal foundation model that supports:
- Zero-shot prediction (no fine-tuning!)
- Few-shot adaptation (minimal data)
- Generative capabilities

In [ ]:
# Create Neuroformer model
neuroformer = NeuroformerModel(
    input_dim=100,
    output_dim=3,
    n_modalities=1,
    task_type='classification'
)

# Pretrain on large dataset
print("Pretraining Neuroformer...")
neuroformer.pretrain(X_neural, n_epochs=20)

print("✅ Pretraining complete!")

### Zero-Shot Prediction

In [ ]:
# Zero-shot: predict without any task-specific training!
zero_shot_preds = neuroformer.zero_shot_predict(
    X_neural[80:85],
    task_description="Classify behavioral state from neural activity"
)

print(f"Zero-shot predictions shape: {zero_shot_preds.shape}")
print(f"Predictions:\n{zero_shot_preds}")
print("\n✅ Zero-shot inference successful (no fine-tuning needed!)")

### Few-Shot Adaptation

In [ ]:
# Few-shot: adapt with only 15 examples (5 per class)
n_shots = 5
support_indices = []
for c in range(3):
    class_indices = np.where(y_behavior[:80] == c)[0][:n_shots]
    support_indices.extend(class_indices)

X_support = X_neural[support_indices]
y_support = y_behavior[support_indices]

# Few-shot adaptation
print(f"Adapting with {len(X_support)} examples ({n_shots} per class)...")
few_shot_preds = neuroformer.few_shot_adapt(
    X_support=X_support,
    y_support=y_support,
    X_query=X_neural[80:],
    n_shots=n_shots
)

# Evaluate
accuracy = accuracy_score(y_behavior[80:], few_shot_preds)
print(f"\nFew-shot accuracy: {accuracy:.2%}")
print(f"(With only {n_shots} examples per class!)")

### Generative Capabilities

In [ ]:
# Generate synthetic neural data
synthetic = neuroformer.generate(
    n_samples=10,
    sequence_length=50
)

print(f"Generated synthetic data: {synthetic.shape}")

# Visualize comparison
fig, axes = plt.subplots(2, 1, figsize=(12, 6))

# Real data
axes[0].imshow(X_neural[0, :, :50].T, aspect='auto', cmap='viridis')
axes[0].set_title('Real Neural Activity')
axes[0].set_xlabel('Time bins')
axes[0].set_ylabel('Neurons')

# Synthetic data
axes[1].imshow(synthetic[0, :, :].T, aspect='auto', cmap='viridis')
axes[1].set_title('Neuroformer Generated Activity')
axes[1].set_xlabel('Time bins')
axes[1].set_ylabel('Neurons')

plt.tight_layout()
plt.show()

## 6. Comparison: Foundation vs. Classical Models

Let's compare foundation models with classical approaches.

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Prepare data for classical models
X_flat = X_neural.reshape(X_neural.shape[0], -1)
X_train_flat, X_test_flat = X_flat[:80], X_flat[80:]
y_train, y_test = y_behavior[:80], y_behavior[80:]

# Classical models
models = {
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(n_estimators=100),
}

results = {}

# Train and evaluate classical models
for name, model in models.items():
    model.fit(X_train_flat, y_train)
    preds = model.predict(X_test_flat)
    acc = accuracy_score(y_test, preds)
    results[name] = acc
    print(f"{name:15s}: {acc:.2%}")

# Foundation model (POYO+)
poyo_acc = accuracy_score(y_behavior[80:], behavior_labels)
results['POYO+'] = poyo_acc
print(f"{'POYO+':15s}: {poyo_acc:.2%}")

# Foundation model (Neuroformer few-shot)
results['Neuroformer'] = accuracy
print(f"{'Neuroformer':15s}: {accuracy:.2%}")

### Visualize Comparison

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

model_names = list(results.keys())
accuracies = list(results.values())
colors = ['gray', 'gray', 'blue', 'blue']

bars = ax.barh(model_names, accuracies, color=colors, alpha=0.7)
ax.set_xlabel('Accuracy')
ax.set_title('Model Comparison: Classical vs. Foundation Models')
ax.set_xlim(0, 1)

# Add value labels
for i, (name, acc) in enumerate(results.items()):
    ax.text(acc + 0.01, i, f'{acc:.1%}', va='center')

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='gray', alpha=0.7, label='Classical'),
    Patch(facecolor='blue', alpha=0.7, label='Foundation')
]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()

## 7. Key Takeaways

### Foundation Models Advantages:

1. **Transfer Learning**: Pretrained on large datasets, work well on small datasets
2. **Multi-Task**: POYO+ handles multiple tasks simultaneously
3. **Cross-Subject**: NDT3 adapts quickly to new subjects
4. **Interpretable**: CEBRA produces consistent latent spaces
5. **Zero-Shot**: Neuroformer works without task-specific training
6. **Few-Shot**: Rapid adaptation with minimal data

### When to Use Each Model:

- **POYO+**: Multiple related tasks, multi-session experiments
- **NDT3**: Real-time motor decoding, cross-subject BCIs
- **CEBRA**: Behavioral analysis, latent space visualization
- **Neuroformer**: New tasks, limited data, generative modeling

### Classical Models Still Useful For:

- Simple, well-defined tasks
- When interpretability is critical
- Fast training on small datasets
- Limited computational resources

## Next Steps

1. **Load Real Data**: Try with NWB files or Allen datasets
2. **Fine-Tune Models**: Use your own pretrained weights
3. **Hyperparameter Tuning**: Optimize for your specific task
4. **Real-Time Inference**: Deploy in production pipeline
5. **Multi-Modal**: Combine neural + behavioral + video data

### Further Reading:

- [Foundation Models User Guide](../user-guide/foundation-models.md)
- [NWB Integration Tutorial](nwb-integration.md)
- [API Reference: Foundation Models](../api/foundation-models.md)
- [Benchmarking Tutorial](benchmarking.md)

## Exercises

1. **Try different CEBRA modes**: Change `learning_mode` to 'behavior' or 'hybrid'
2. **Experiment with latent dimensions**: Try 2D, 8D, 32D for CEBRA
3. **Test cross-subject transfer**: Train NDT3 on subject 1, test on subject 2
4. **Compare few-shot performance**: Try 1, 3, 5, 10 shots per class
5. **Generate conditioned data**: Use Neuroformer with context

---

**Congratulations!** You've learned how to use all four foundation models in NeurOS. 🎉